# **MENTR**

## 1. CPUに関する確認

初回実行時のみ必要。

### 1.1 CPUアーキテクチャの確認

次の出力が arm64 であることを確認する。

In [1]:
!uname -m

arm64


### 1.2 CPU数（スレッド数)の確認

次の数を確認し、後述の `n_cpu` で指定する。

In [2]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

## 2. MENTRのセットアップ

初回実行時のみ必要。

### 2.1. MENTRのスクリプトと学習済モデルのダウンロード

MENTRをセットアップするディレクトリに移動（任意）。

In [3]:
cd /tmp

/private/tmp


`git` を用いてMENTR一式をダウンロード

In [5]:
!git clone https://github.com/koido/MENTR

Cloning into 'MENTR'...
remote: Enumerating objects: 786, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 786 (delta 22), reused 34 (delta 13), pack-reused 730
Receiving objects: 100% (786/786), 190.30 MiB | 18.54 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Updating files: 100% (730/730), done.


ダウンロードしたMENTRディレクトリに移動

In [6]:
cd MENTR

/private/tmp/MENTR


### 2.2. MENTR依存ライブラリのインストールする

数分要する。

In [9]:
%%bash
CONDA_SUBDIR=osx-64 conda create -y -n mentr python=3.6.4
eval "$(conda shell.bash hook)" 
conda activate mentr
conda config --env --set subdir osx-64
conda install -y -c mkoido -c pytorch -c conda-forge -c bioconda --file mentr_env.txt

Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/koido/src/anaconda3.2022.10.arm64/envs/mentr_dev

  added / updated specs:
    - python=3.6.4


The following NEW packages will be INSTALLED:

  ca-certificates    pkgs/main/osx-64::ca-certificates-2023.01.10-hecd8cb5_0 
  certifi            pkgs/main/osx-64::certifi-2021.5.30-py36hecd8cb5_0 
  libcxx             pkgs/main/osx-64::libcxx-14.0.6-h9765a3e_0 
  libedit            pkgs/main/osx-64::libedit-3.1.20221030-h6c40b1e_0 
  libffi             pkgs/main/osx-64::libffi-3.2.1-h0a44026_1007 
  ncurses            pkgs/main/osx-64::ncurses-6.4-hcec6c5f_0 
  openssl            pkgs/main/osx-64::openssl-1.0.2u-h1de35cc_0 
  pip                pkgs/main/osx-64::pip-21.2.2-py36hecd8cb5_0 
  python             pkgs/main/osx-64::python-3.6.4-hc167b69_3 
  readline           pkgs/mai

### 2.3. External Dependencyのダウンロード

30分程度要する場合がある。

In [ ]:
!wget http://deepsea.princeton.edu/media/code/expecto/resources.tar.gz
!tar xfz resources.tar.gz -C ./

--2023-03-27 15:17:18--  http://deepsea.princeton.edu/media/code/expecto/resources.tar.gz
Resolving deepsea.princeton.edu (deepsea.princeton.edu)... 128.112.117.108
Connecting to deepsea.princeton.edu (deepsea.princeton.edu)|128.112.117.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6643086006 (6.2G) [application/x-gzip]
Saving to: 'resources.tar.gz'

resources.tar.gz     12%[=>                  ] 800.67M  5.42MB/s    eta 21m 21s

## 3. MENTR *in silico*変異導入法の実行

本デモではtarget CAGE IDを `-p`オプションで指定している。

`-p`オプションを用いない場合、各バリアントとFANTOM5の全CAGE IDの距離を計算し、前後100kb (default) のペアを探索する（1染色体あたり30分程度要する可能性がある）。

この場合、各自の計算機環境に応じて分散処理をするなど、shellスクリプトの改造をおすすめする。

### 3.1 実行

macOSでの実行例を示している。

In [ ]:
%%bash
eval "$(conda shell.bash hook)" 
conda activate mentr

# alias for macOS
zcat() { gzcat "$@"; }
export -f zcat

# output directory
cmn_dir=${PWD}/demo
rm -Rf ${cmn_dir}
mkdir -p ${cmn_dir}

# input file (example: rs12637953)
in_f=${PWD}/resources/example/rs12637953.txt

# target CAGE ID
target_CAGE_ID="chr3:112395778-112395997"

# DeepSEA Beluga
deepsea=${PWD}/resources/deepsea.beluga.2002.cpu

# hg19
hg19_fa=${PWD}/resources/hg19.fa

# Window size [default=100000]
w_size=1000

# N of CPUs (threads)
n_cpu=10

# run
bash bin/quick.mutgen.sh -i ${in_f} -o ${cmn_dir} -m ${deepsea} -f ${hg19_fa} -w ${w_size} -t ${n_cpu} -p ${target_CAGE_ID} -c

### 3.2 結果の確認

In [ ]:
import pandas as pd

# summarized result
in_f = 'demo/output/all_qcd_res.txt.gz'

# specify dtype
df_h = pd.read_csv(in_f, sep='\t', header=None, dtype = 'str', nrows=1).values.reshape(-1).tolist()
df_dtypes = {}
for x in df_h:
    if x in ['CHR', 'REF', 'ALT', 'gene', 'strand']:
        df_dtypes[x] = 'str'
    elif x in ['POS', 'dist']:
        df_dtypes[x] = 'int64'
    elif x in ['ref_matched']:
        df_dtypes[x] = 'bool'
    else:
        df_dtypes[x] = 'float64'

# read table        
df = pd.read_csv(in_f, sep='\t', dtype=df_dtypes)
df